In [1]:
import pandas as pd
import numpy as np
import scipy.optimize as optimize



In [2]:
par = 1000


In [3]:



def bond_ytm(price, par, T, coup, freq=2, guess=0.05):
    freq = float(freq)
    periods = T*freq
    coupon = coup/100.*par/freq
    dt = [(i+1)/freq for i in range(int(periods))]
    ytm_func = lambda y: sum([coupon/(1+y/freq)**(freq*t) for t in dt]) + par/(1+y/freq)**(freq*T) - price
        
        
        
    return optimize.newton(ytm_func, guess)


ytm = bond_ytm(1000, 1000, 10, 20, 2)
ytm

0.19999999999999996

In [4]:
def bond_price(par, T, ytm, coup, freq=2):
    freq = float(freq)
    periods = T*freq
    coupon = (coup/100)*par/freq
    dt = [(i+1)/freq for i in range(int(periods))]
    price = sum([coupon/(1+ytm/freq)**(freq*t) for t in dt]) + \
            par/(1+ytm/freq)**(freq*T)
    return price


pv = bond_price(1000, 10, 0.15, 15)
pv

1000.0000000000003

In [5]:
def bond_fv(price, T, ytm, coup, freq=2):
    periods = T * freq
    coupon_payment = coup / freq
    fv = 0

    for i in range(1, periods + 1):
        fv = coupon_payment / ((1 + ytm**0.5) ** i)
    vf = fv+1000

   

    return vf

bond_fv(5, 5, 0.1, 20)

1000.6407513344584

In [26]:
def bond_macaulay_d(y, c, h, n):
    dura = (1+y/y)*h+(y-c/y)*n*(1-h)
    return dura

bond_macaulay_d(1.05, 0.05, 0.6, 10)

5.20952380952381

In [7]:
def effective_duration(price, par, T, coup, freq=2, dy=0.01):
    ytm = bond_ytm(price, par, T, coup)
    
    ytm_minus = ytm - dy   
    price_minus = bond_price(par, T, ytm_minus, coup, freq)
    
    ytm_plus = ytm + dy
    price_plus = bond_price(par, T, ytm_plus, coup, freq)
    
    mduration = (price_minus-price_plus)/(2*price*dy)
    return mduration



In [8]:
def bond_convexity(price, par, T, coup, freq=2, dy=0.01):
    ytm = bond_ytm(price, par, T, coup, freq)

    ytm_minus = ytm - dy    
    price_minus = bond_price(par, T, ytm_minus, coup, freq)
    
    ytm_plus = ytm + dy
    price_plus = bond_price(par, T, ytm_plus, coup, freq)
    
    convexity = (price_minus+price_plus-2*price)/(price*dy**2)
    return convexity

bond_convexity(1000, 1000, 10, 50, 2)


7.541082988354901

In [9]:
#Sistema para gerar titulos 
cdi_gen = np.random.uniform(0, 0.20)

cdif = [cdi_gen]

for i in range(9):
    cdi_gen = cdi_gen * (1 + np.random.uniform(-0.15, 0.15))
    cdif.append(cdi_gen)


t_anos = 10

anos = [pd.Timestamp(year=i+2024, month=1, day=1) for i in range(t_anos)]


cdi = pd.DataFrame({"CDI Futuro" : cdif, "Ano" : anos})

cdi['Ano'] = pd.to_datetime(cdi['Ano'])

cdi['Ano'] = cdi['Ano'].dt.year.astype(str)

cdi.set_index("Ano")

    
    

,CDI Futuro
Ano,
2024,0.009314
2025,0.009056
2026,0.008703
2027,0.009796
2028,0.008800
2029,0.008829
2030,0.007531
2031,0.007510
2032,0.008096


In [10]:
class bonds:
    def __init__(self):
        self.par_value = 1000
        self.coupon = np.random.randint(low=0, high= self.par_value*0.05)
        self.maturity = np.random.randint(2024, 2033)
        self.vencimento = self.maturity - 2023
        ano_maturidade = str(self.maturity)
        
        
        
        
        for index, row in cdi.iterrows():
            if row['Ano'] == ano_maturidade:
                self.ytm_init = row['CDI Futuro']
                break
        
        self.pv = bond_price(self.par_value, self.vencimento,self.ytm_init , self.coupon)
        self.efecdu = effective_duration(self.pv, self.par_value, self.vencimento, self.coupon)
        self.convex = bond_convexity(self.pv, self.par_value,self.vencimento ,self.coupon )
    def risco_buy(self):
        teste = np.random.randint(low=0, high= 3)
        if teste == 0:
            self.ytm_init = self.ytm_init + np.random.uniform(low=0, high= 0.01)
        elif teste == 1:
            self.ytm_init = self.ytm_init + np.random.uniform(low=0.01, high= 0.05)
        elif teste == 2:
            self.ytm_init = self.ytm_init + np.random.uniform(low=0.05, high= 0.1)

    def risco_sell(self):
        teste = np.random.randint(low=0, high= 3)
        if teste == 0:
            self.ytm_init = self.ytm_init - np.random.uniform(low=0, high= self.ytm_init*0.1)
        elif teste == 1:
            self.ytm_init = self.ytm_init - np.random.uniform(low=self.ytm_init*0.1, high= self.ytm_init*0.2)
        elif teste == 2:
            self.ytm_init = self.ytm_init - np.random.uniform(low=self.ytm_init*0.2, high= self.ytm_init*0.3)
        




In [11]:
def create_bonds():
    data = []
    for i in range(len(cdi["CDI Futuro"])):
        bond = bonds()
        bond.risco_buy()
        title = f"Title {i+1}"
        pv = round(bond.pv, 2)
        ytm = round(bond.ytm_init*100, 2)
        vencimento = (bond.vencimento)
        cupom = (bond.coupon)
        effec = (bond.efecdu)
        convex = (bond.convex)
        data.append({"Nome": title, "Valor Presente": pv, "YTM" : ytm, "Vencimento" : vencimento, "Taxa de Cupom" : cupom, "Duration Efetiva": effec, "Convexidade":convex})
    df = pd.DataFrame(data)
    df.set_index("Nome", inplace=True)
    return df.sort_values(by="Vencimento", ascending=True)

create_bonds()

,Valor Presente,YTM,Vencimento,Taxa de Cupom,Duration Efetiva,Convexidade
Nome,,,,,,
Title 1,1338.32,1.59,1,35,0.930633,1.357230
Title 3,1456.71,7.18,2,24,1.747012,4.145431
Title 5,2126.67,3.02,3,39,2.309634,7.262999
Title 6,2722.63,1.75,4,45,2.846734,11.060897
Title 7,1249.91,7.18,5,6,4.451268,23.409365
Title 9,3455.98,1.18,6,43,3.970430,21.393325
Title 2,2922.54,1.03,7,29,4.770116,30.172646
Title 4,4079.60,3.42,7,46,4.468759,27.178435
Title 8,3731.91,8.04,8,36,5.152014,35.710265


In [12]:
def bond_portfolio(dados):
    data = []
    name = "Carteira de titulos ativos"
    df_base= dados
    pv = df_base["Valor Presente"].sum()

    
    data.append({"Nome": name, "Valor Presente" : pv})
    df = pd.DataFrame(data)

    return df.set_index("Nome")

bond_portfolio()

TypeError: bond_portfolio() missing 1 required positional argument: 'dados'